In [9]:
import pandas as pd
import csv
#import geopandas as gp

What does MANAGING_ORG mean in the header? Is that the ranger district where the structure is located?
MANAGING_ORG identifies the district responsible for administration of the authorized use, most likely it is also occurring on that district or forest. It also shows up in some reports as ‘RRFFDD.’
It identifies the region, the forest, and the district.  Sometimes the number is displayed as a six-digit number and sometimes its displayed as a five-digit number.  That’s a result of Excel autoformatting numbers.
Either way, the first two numbers, if six, are the region. If it’s five, the first number is the region. The next two are the forest and the last two are the district. 
                                                               i.      050357 = Region 5, Forest 3, District 57
                                                             ii.      0304 = Region 3, Forest 4
                                                           iii.      21301 = Region 2, Forest 13, District 1

# Read the original permit file, call it 'original'

In [3]:
original = pd.read_excel('original water use file.xlsx', dtype={'CN': str, 'RRFFDD': str})
#split up region, forest and district
original['region'] = original['RRFFDD'].str[:2]
original['forest'] = original['RRFFDD'].str[2:4]
original['district'] = original['RRFFDD'].str[4:6]
original['rrffdd_count'] = original['RRFFDD'].str.len()
#check to see how many complete numbers we have 
#stringcount = original.groupby('rrffdd_count')['rrffdd_count'].count().to_frame(name = 'count').reset_index()
#stringcount.sort_values('count', ascending=False).head(20)

In [8]:
#there are four permits that belong to regions that don't exist
lengthcheck = original.groupby('rrffdd_count')['rrffdd_count'].count().to_frame(name = 'count').reset_index()
lengthcheck.sort_values('count', ascending=False).head(20)

,rrffdd_count,count
2,6,17340
1,4,926
0,2,3


# Read the ranger district shapefile as 'districts'

In [112]:
zipfile = "zip:///Users/cmcglade/Downloads/S_USA.RangerDistrict.zip"
districts = gp.read_file(zipfile)
districts = districts.to_crs('EPSG:5070')

# Merge the district file with the original permit data, call it 'districtmerge'
1,406 out of 18,269 permits did not match a district

In [113]:
districtmerge = original.merge(districts, left_on='RRFFDD', right_on="DISTRICTOR",how="left")
districtmerge = districtmerge[['CN', 'RRFFDD', 'ID', 'DISTRICTNA', 'STATUS', 'ISSUE_DATE', 'USE1_NAME', 'USE2_NAME', 'USE3_NAME', 'EXP_EXPIRATION_DATE', 'CONTACT_TYPE', 'CONTACT_NAME', 'PREVIOUS_AUTH', 'region', 'forest', 'district', 'rrffdd_count']]
districtmerge.to_csv('districtmerge.csv')

In [114]:
districtmerge.shape

(18269, 17)

# Select forests from the district shapefile, merge districtmerge to forest names to capture the permits that didn't have a district number
Only 129 permits out of 18,269 did not match a forest.  

In [115]:
forests = districts[['REGION', 'FORESTNUMB', 'FORESTNAME']]
forests = forests.drop_duplicates()
forestmerge = pd.merge(districtmerge, forests,  how='left', left_on=['region','forest'], right_on = ['REGION','FORESTNUMB'])
forestmerge.to_csv('permits_master_1.csv')

In [116]:
active_permits =  forestmerge[forestmerge['STATUS'] == 'ISSUED']
active_permits.shape
#dupecheck = active_permits.groupby('ID')['ID'].count().to_frame(name = 'count').reset_index()
#dupecheck.sort_values('count', ascending=False).head(20)

(7615, 20)

In [117]:
forestmerge = forestmerge[['CN', 'RRFFDD', 'ID', 'STATUS', 'ISSUE_DATE', 'USE1_NAME',
       'USE2_NAME', 'USE3_NAME', 'EXP_EXPIRATION_DATE', 'CONTACT_TYPE',
       'CONTACT_NAME', 'PREVIOUS_AUTH', 'region', 'forest', 'district',
       'rrffdd_count', 'FORESTNAME', 'DISTRICTNA']]

# DATA DICTIONARY
CN = unique ID for every permit

RRFFDD = Region, Forest, District ID

ID = unique ID for each permit within a ranger district. 

STATUS = the status of the permit. 'Issued' are the current ones. 

USE1_NAME = One of the structures that this permit covers

Use2_NAME = Another structure taht this permit covers

Use3_NAME = A third structure that this permit covers

ISSUE_DATE = The date the permit was issued 

EXP_EXPIRATION_DATE = The date the permit expires

CONTACT_TYPE = Permittee contact type

CONTACT_NAME = Permittee contact name

region = the region number where the permit is located

forest = the forest number where the permit is located

district = the ranger district number where the permit is located 

rrffdd_count = the number of characters in the RRFFDD field so that I could tell which ones definitely wouldn't join on district 

FORESTNAME = the name of the forest where the permit is located

DISTRICTNA = The name of the ranger district where the permit is located